In [2]:
#arcpy to process LIDAR data    

import arcpy
from arcpy import env

In [3]:
#define variables

#folder that contains las data
las_folder = r"K:\DataServices\Datasets\MassGIS\LiDAR\Revere pilot\LAS Data"

#dataset file path
las_dataset = r"K:\DataServices\Datasets\MassGIS\LiDAR\Revere pilot\LAS Data\Revere pilot_LasDataset.lasd"

point_selection_method = 'CLOSEST_TO_CENTER'

env.workspace = "C:/data"


In [5]:
if arcpy.Exists (las_dataset): 
    arcpy.Delete_management (las_dataset)

arcpy.CreateLasDataset_management(input=las_folder,
                                  out_las_dataset=las_dataset,
                                  compute_stats='COMPUTE_STATS')
                                  

<Result 'K:\\DataServices\\Datasets\\MassGIS\\LiDAR\\Revere pilot\\LAS Data\\Revere pilot_LasDataset.lasd'>

In [8]:
arcpy.management.BuildLasDatasetPyramid(in_las_dataset=las_dataset,
                                 point_selection_method=point_selection_method)

ExecuteError: ERROR 050157: Failed to process the LAS dataset.
The 3D Analyst extension has not been enabled.
The 3D Analyst extension has not been enabled.
Failed to execute (BuildLasDatasetPyramid).
